In [3]:
from sklearn.datasets import load_iris
from sklearn.model_selection import train_test_split
from keras.utils import to_categorical
import tensorflow as tf
import numpy as np

x = load_iris().data
y = load_iris().target

Using TensorFlow backend.


In [0]:
y = to_categorical(y)
x_train, x_test, y_train, y_test = train_test_split(x, y, test_size=0.20)

with tf.name_scope('input'):
  input_features = tf.placeholder(dtype=tf.float32, shape=[None, x.shape[1]], name='input_features')
  input_labels = tf.placeholder(dtype=tf.float32, shape=[None, x.shape[1]], name='input_labels')

  with tf.name_scope('model'):
    weights = tf.Variable(tf.random_normal(shape=[x.shape[1], y.shape[1]]))
    biases = tf.Variable(tf.random_normal(shape=[y.shape[1]]))
    linear_model = tf.add(tf.matmul(input_features, weights), biases)
    predictions = tf.nn.softmax(linear_model)
                                

In [0]:
with tf.name_scope('training_ops'):
  loss = tf.reduce_mean(tf.nn.softmax_cross_entropy_with_logits_v2(
          logits=linear_model, labels=input_labels))
  train_op = tf.train.GradientDescentOptimizer(learning_rate=0.001).minimize(loss)
  tf.summary.scalar(name='loss', tensor=loss)
  
with tf.name_scope('metrics'):
  correction_prediction = tf.cast(tf.equal(tf.argmax(predictions, 1), tf.argmax(input_labels, 1)), tf.float32)
  accuracy_op = tf.reduce_mean(correction_prediction)
  tf.summary.scalar(name='accuracy', tensor=accuracy_op)


In [0]:
init_op = tf.global_variables_initializer()

In [0]:
def next_batch(batch_size, features, labels):
  indices = np.arange(start=0, stop=features.shape[0])
  np.random.shuffle(indices)
  indices = indices[:batch_size]
  return features[indices], labels[indices]


In [8]:
!rm -rf logs/
!rm ngrok*
!wget https://bin.equinox.io/c/4VmDzA7iaHb/ngrok-stable-linux-amd64.zip
!unzip ngrok-stable-linux-amd64.zip

--2018-08-17 04:26:20--  https://bin.equinox.io/c/4VmDzA7iaHb/ngrok-stable-linux-amd64.zip
Resolving bin.equinox.io (bin.equinox.io)... 52.44.144.199, 52.22.213.157, 52.23.126.223, ...
Connecting to bin.equinox.io (bin.equinox.io)|52.44.144.199|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 5363700 (5.1M) [application/octet-stream]
Saving to: ‘ngrok-stable-linux-amd64.zip’

ngrok-stable-linux- 100%[===================>]   5.11M  8.64MB/s    in 0.6s    

2018-08-17 04:26:21 (8.64 MB/s) - ‘ngrok-stable-linux-amd64.zip’ saved [5363700/5363700]

Archive:  ngrok-stable-linux-amd64.zip
  inflating: ngrok                   


In [0]:
LOG_DIR = './logs'
get_ipython().system_raw(
  'tensorboard --logdir {} --host 0.0.0.0 --port 6006 &'
   .format(LOG_DIR)
)

In [0]:
get_ipython().system_raw('./ngrok http 6006 &')

In [11]:
! curl -s http://localhost:4040/api/tunnels | python3 -c \
    "import sys, json; print(json.load(sys.stdin)['tunnels'][0]['public_url'])"

http://cf190af4.ngrok.io


In [14]:
with tf.Session() as sess:
  sess.run(init_op)
  
  #epochs = 10
  
  for epoch in range(10):
    for mini_batch in range(int(x_train.shape[0] / 16)):
      batch_x, batch_y = next_batch(batch_size=16, features=x_train, labels=y_train)
      
      _, train_loss, train_accuracy, train_summary = sess.run([train_op, loss, accuracy_op, summary],
                                              feed_dict={input_features: batch_x,
                                                        input_labels: batch_y})
      writer.add_summary(summary=train_summary, global_step=mini_batch)
      print('Epoch {}, loss: {}, accuracy: {}'.format(epoch, train_loss,
                                                     train_accuracy))
  test_accuracy = sess.run(accuracy_op, feed_dict={input_features: x_test, input_labels: y_test})
  print('Test accuracy : {}'.format(test_accuracy))

NameError: ignored

In [0]:
with tf.Session() as sess:
  sess.run(init_op)
  prediction_ = sess.run(predictions, feed_dict={input_features: x_test[1].reshape(-1, x_test.shape[1])})
  print(prediction_)
  print('predicted class : {}, true class : {}'.format(sess.run(tf.argmax(prediction_, 1)), sess.run(tf.argmax(y_test[1]))))